## 1 - Objetivo

A intenção deste exercício é aprender alguns recursos avançados do scikit-learn, especialmente como utilizar *Cross validation*, *Feature Selection* e *Grid Search* para otimizar os hyperparâmetros/features do modelo. Também vamos utilizar este exercício para nos familiarizarmos com as métricas padrão de avaliação de resultados.



## 2 - Carregando as bibliotecas

Mais uma vez vamos utilizar o Scikit-learn. Devido a sua facilidade de utilização e métodos pré-implementados é fácil ver o motivo de ter se tornado padrão na indústria de tecnologia.

Para utilizá-la, vamos primeiro carregar os métodos/módulos necessários

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt  # primeiro importamos a biblioteca para visualização
import numpy as np  # importamos também a biblioteca NumPy que irá nos fornecer diversos métodos para trabalhar com arrays
import seaborn as sns

#### 2.1 - Funções auxiliares

De maneira geral, os algoritmos de machine learning precisam de features numéricas. Por isso, precisamos converter as features que estão em formato textual para um formato mais apropriado. A função abaixo faz isso utilizando uma função pré-implementada pela biblioteca ```pandas```.


In [ ]:
def converte_categorias(df):
    pd.options.mode.chained_assignment = None  # default='warn'
    # job
    df.job = pd.Categorical(df.job)
    df['job'] = df.job.cat.codes
    # marital
    df.marital = pd.Categorical(df.marital)
    df['marital'] = df.marital.cat.codes
    # education
    df.education = pd.Categorical(df.education)
    df['education'] = df.education.cat.codes
    # default
    df.default = pd.Categorical(df.default)
    df['default'] = df.default.cat.codes
    # housing
    df.housing = pd.Categorical(df.housing)
    df['housing'] = df.housing.cat.codes
    # loan
    df.loan = pd.Categorical(df.loan)
    df['loan'] = df.loan.cat.codes
    # contact
    df.contact = pd.Categorical(df.contact)
    df['contact'] = df.contact.cat.codes
    # month
    df.month = pd.Categorical(df.month)
    df['month'] = df.month.cat.codes
    # outcome
    df.poutcome = pd.Categorical(df.poutcome)
    df['poutcome'] = df.poutcome.cat.codes
    return df

#### 3 - Dataset

O dataset que vamos utilizar aqui foi disponibilizado para o *UCI Machine Learning repository* por Moro et al., 2014 [1]. Este é um dataset criado para uma campanha de marketing bancário em 2011. Para mais detalhes sobre o dataset, acesse [este link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).

*Observação*: O *UCI Machine Learning repository* é o maior repositório de datasets para experimentos com Machine Learning. Ele contém centenas de datasets para download assim como as fontes que os construíram. Desta forma, podemos comparar a performance dos nossos experimentos com a performance obtida pelos criadores do dataset. É um excelente lugar para praticar nossas habilidades com o machine learning.


Abaixo nós utilizamos a biblioteca ```pandas``` para criar um objeto ```DataFrame``` contendo os datapoints.

<sup>[1] S. Moro, P. Cortez and P. Rita. *A Data-Driven Approach to Predict the Success of Bank Telemarketing*. Decision Support Systems, Elsevier, 62:22-31, June 2014</sup>

In [ ]:
trainset = pd.read_csv("data/trainset.csv")
testset = pd.read_csv("data/testset.csv")

X_train = trainset.loc[:, trainset.columns != "y"]
y_train = trainset.loc[:, trainset.columns == "y"]
y_train = y_train.values.ravel()


X_test = testset.loc[:, testset.columns != "y"]
y_test = testset.loc[:, testset.columns == "y"]
y_test = y_test.values.ravel()

trainset

Devemos sempre visualizar os nossos datapoints para termos uma ideia de com o que estamos lidando. No nosso caso, vamos utilizar uma classe da biblioteca ```seaborn``` para criar os gráficos. Entretanto, na maioria dos casos o tamanho do dataset torna proibitivo o uso de algumas funções auxiliares das bibliotecas de visualização e devemos criar os gráficos de forma independente.

In [ ]:
sns.pairplot(trainset, diag_kind='kde', hue="y")  # o parametro 'hue' diz qual coluna contém o alvo para distribuir as cores


#### 3.1 - Convertendo as features

Como mencionado anteriormente, alguns algoritmos de machine learning precisam que as features estejam em formato numérico. Abaixo utilizamos a função que criamos no início do exercício para realizar a conversão.

In [ ]:
X_train = converte_categorias(X_train)
X_test = converte_categorias(X_test)
X_train

### 4 - Experimentos

Feita a conversão das features é hora de realizarmos nossos experimentos.

#### 4.1 - Carregando as bibliotecas necessárias

Como mencionado anteriormente, o ScikitLearn possui diversas funções auxiliares para realizarmos experimentos e encontrar o melhor modelo para uma determinada tarefa. Abaixo nós importaremos as funções relevantes e um algoritmo para realizarmos experimentos.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


# from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
from sklearn.metrics import classification_report

#### 4.2 - Buscando os melhores hyperparâmetros 

Agora que as funções estão carregadas, vamos testá-las e gerar o melhor resultado possível para nossa tarefa. 


In [ ]:
folds = 3  # precisa ser maior que 2

# aqui nós criamos um objeto dicionário com o nome do hyperparâmetro e os possíveis valores que ele pode assumir
# os hyperparâmetros são específicos para cada algoritmo! Note como neste caso nós chamamos a classe do algoritmo
# sem passarmos os hyperparâmetros default
knn = KNeighborsClassifier()
knn_params = {
    "n_neighbors": [2, 3, 5, 10],
    "weights": ["uniform", "distance"],
    "p": [1, 2]
}

# O scikit learn e uma biblioteca bastante flexível. Com poucas linhas de código, podemos executar o mesmo experimento
# para diversos algoritmos: basta criar uma lista contendo os algoritmos e uma segunda lista contendo os 
# dicionários de hyperparâmetros. Detalhe: a ordem das listas é importante!
classifiers = [knn]
grids = [knn_params]

# gera uma lista de tuplas entre classifiers e grids para que cada um fique na
# posição correta [(class.1, parms.1), (class.2, params.2), ...]
grid_params = zip(classifiers, grids)

# aqui fazemos a busca - neste caso a busca é por força bruta, ou seja, vai
# testar todas as combinações que incluirmos no dicionário de parâmetros - há
# também a opção de se buscar randomicamente, mas precisariamos definir
# distribuições ao invés de parâmetros e os resultados são parecidos.
# a busca vai ser feita pelo ``score'' que definirmos

for _, (classifier, params) in enumerate(grid_params):

    print("Buscando para aloritmo: {0}\n".format(classifier.__class__))

    clf = GridSearchCV(estimator=classifier,  # algoritmo em teste
                               param_grid=params,  # parâmetros de busca
                               cv=folds,  # objeto que vai gerar as divisões
                               n_jobs=-1,
                               scoring='accuracy')  # score que será utilizado
    clf.fit(X_train, y_train.ravel())

    # aqui nós imprimimos o resultado - o método report vai imprimir as ``top''
    # melhores combinações encontrada na busca. Os parâmetros impressos
    # são aqueles que teríamos que usar para gerar o classificador de forma isolada
    print("Melhor seleção de hyperparâmetros:\n")
    print(clf.best_params_)
    print("\nScores (% de acertos) nos folds de validação:\n")
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("{:.3f} (+/-{:.3f}) for {}".format(mean, std * 2, params))

    print("\nResultado detalhado para o melhor modelo:\n")
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))

Observe a variação em cada um dos folds do experimento. Esta ocorrência deve-se ao fato de termos um dataset bastante diverso. Isso demonstra a importância de realizarmos o *Cross Validation* para termos uma ideia melhor do comportamento do algoritmo!

### 5 - Exercícios

#### 5.1 - Experimentando com outros algoritmos

Agora que vimos como funciona, vamos experimentar outros algoritmos. Verifique na [documentação do ScikitLearn](http://scikit-learn.org/stable/modules/classes.html) e tente otimizar hyperparâmetros de outros algoritmos. Na célula de código do ítem 4.1 estão algumas sugestões (em forma de comentário) de algoritmos.

Rode experimentos com diferentes algortimos e note como os resultados variam. Note também como alguns algoritmos são mais rápidos para treinar enquanto alguns demoram um pouco mais. Paciência é uma virtude que todo praticante de machine learning deve aprender desde o início!


#### 5.2 - Desafio

Nesta versão da busca por melhores hyperparâmetros nós deixamos de fora a busca pelo melhor conjunto de features. Modifique o código acima baseando-se na explicação [deste link](http://scikit-learn.org/stable/modules/feature_selection.html) usando o código [deste tutorial](http://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_with_cross_validation.html#sphx-glr-auto-examples-feature-selection-plot-rfe-with-cross-validation-py) e implemente a seleção de features.